In [1]:
from jobs import SRjobs, SRfeatures
import pandas as pd
schedule_mstr = SRjobs.get_season_played_games()
future_mstr = SRjobs.get_season_future_games()
history = SRjobs.get_history()

Initiate get_season_played_games
Finished scraping 2018 schedule with 845 rows
Successfully loaded 828.0 played games from 2018
Initiate get_season_future_games
Finished scraping 2018 schedule with 845 rows
Successfully loaded future_mstr with 17 remaining games in 2018


In [2]:
full_history = pd.concat([history, schedule_mstr])
print('Applying features to games already played')
full_history = SRfeatures.apply_features(full_history)
all_time = pd.concat([full_history, future_mstr], sort=True)
print('Applying features to future games')
future_mstr = SRfeatures.apply_future_features(future_mstr, all_time)

Applying features to games already played
Added Game_conf
Added Team_SRTD
Added Team_CRTD
Added Opp_SRTD
Added Opp_CRTD
Applying features to future games
Added Game_conf
Added Team_SRTD
Added Team_CRTD
Added Opp_SRTD
Added Opp_CRTD


In [5]:
full_history.to_csv('./SRinput/current/past_games.csv')

In [4]:
future_mstr.to_csv('./SRinput/current/future_games.csv')